In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/bundesliga-top-7-teams-offensive-stats/bundesliga_top7_offensive.csv')

In [ ]:
df.head()

In [ ]:
df=df.replace({'CHI':'CHL','CRO':'HRV','DEN':'DNK','ENG':'GBR','GER':'DEU','GUI':'GIN','NED':'NLD','POR':'PRT','SUI':'CHE'})

# 1. Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

**1) Nationality**

In [ ]:
df1=pd.DataFrame(df.groupby('Nationality')['Name'].count())
df1=df1.reset_index()

In [ ]:
import matplotlib.pyplot as plt
import plotly_express as px
fig = px.choropleth(df1,locations='Nationality',color='Name',scope='world')
fig.show()

**2) Goals by Nationality,Position and Minutes**

In [ ]:
plt.figure(figsize=(8, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Nationality'],y=df['Goals'],hue=df['Position'],size=df['Mins'],data=df,sizes=(1,150))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

**3) Goals by Age,Position and Minutes**

In [ ]:
plt.figure(figsize=(8, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Age'],y=df['Goals'],hue=df['Position'],size=df['Mins'],data=df,sizes=(1,150))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

**4) Assists by Nationality,Position and Minutes**

In [ ]:
plt.figure(figsize=(8, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Nationality'],y=df['Assists'],hue=df['Position'],size=df['Mins'],data=df,sizes=(1,150))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

**5) Assists by Age,Position and Minutes**

In [ ]:
plt.figure(figsize=(8, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Age'],y=df['Assists'],hue=df['Position'],size=df['Mins'],data=df,sizes=(1,150))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

**6) Goals by Club and Position**

In [ ]:
plt.figure(figsize=(8, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Club'],y=df['Goals'],hue=df['Position'],size=df['Goals'],data=df,sizes=(1,150))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

**7) Assists by Club and Position**

In [ ]:
plt.figure(figsize=(8, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Club'],y=df['Assists'],hue=df['Position'],size=df['Assists'],data=df,sizes=(1,150))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

**8)Efficient Goals**

In [ ]:
df['Mins/Goals']=df['Mins']/df['Goals']
df['Mins/Goals']=pd.DataFrame(df['Mins/Goals'].replace([np.inf, -np.inf], np.nan))
df_eff=df.dropna(how='any')
df_eff1 = df_eff.sort_values('Goals', ascending=False).head(30)

In [ ]:
columns=df_eff1['Name']
y1 = df_eff1['Mins/Goals']
y2 = df_eff1['Goals']

plt.rcParams["figure.figsize"] = (20, 8)

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(columns, y1, linewidth=2, color="red", linestyle="solid", marker="o", markersize=8, label='Mins/Goals')
ax2.bar(columns, y2, label='Goals')

ax2.set_ylim([0,50])

ax1.set_zorder(2)
ax2.set_zorder(1)

ax1.patch.set_alpha(0)
ax1.legend(bbox_to_anchor=(0, 1), loc='upper left', borderaxespad=0.5, fontsize=10)
ax2.legend(bbox_to_anchor=(0, 0.9), loc='upper left', borderaxespad=0.5, fontsize=10)

ax1.grid(True)

plt.xlabel('Name')
ax1.set_ylabel('Mins/Goals')
ax2.set_ylabel('Goals')
ax1.set_xticklabels(df_eff1["Name"], rotation=90)

plt.show()

# 2. Correlation

In [ ]:
df1=df.drop(['Name','xG','xA','Penalty_Goals','Penalty_Attempted'],axis=1)

In [ ]:
df2=pd.get_dummies(df1)

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
df2.corr().head()

In [ ]:
df2=df2.rename(columns={'Position_DF,FW': 'Position_DFFW', 'Position_DF,MF': 'Position_DFMF','Position_FW,DF':'Position_FWDF','Position_FW,MF':'Position_FWMF','Position_MF,DF':'Position_MFDF','Position_MF,FW':'Position_MFFW'})

In [ ]:
plt.figure(figsize=(15, 12))
sns.heatmap(df2.corr())

# 3.Goals Prediction Model

In [ ]:
from sklearn.linear_model import Ridge,Lasso,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

**1) Multiple regression analysis**

In [ ]:
x=df2.drop(['Goals','Mins/Goals'],axis=1)
y=df2['Goals']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x, y,train_size=0.8,random_state=123)

**1)-1 LR, Lasso and Ridge**

In [ ]:
model1=LinearRegression()
model2=Lasso(alpha=1.0)
model3=Ridge(alpha=0.1)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train.iloc[tr_idx],x_train.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model1.fit(tr_x,tr_y)
    va_pred1=model1.predict(va_x)
    score_rmse1=np.sqrt(mean_squared_error(va_y,va_pred1)).mean()
    score_mae1=mean_absolute_error(va_y,va_pred1).mean()
    score_r21=r2_score(va_y,va_pred1).mean()

In [ ]:
print('rmse1:',score_rmse1)
print('mae1:',score_mae1)
print('R21:',score_r21)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train.iloc[tr_idx],x_train.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model2.fit(tr_x,tr_y)
    va_pred2=model2.predict(va_x)
    score_rmse2=np.sqrt(mean_squared_error(va_y,va_pred2)).mean()
    score_mae2=mean_absolute_error(va_y,va_pred2).mean()
    score_r22=r2_score(va_y,va_pred2).mean()

In [ ]:
print('rmse2:',score_rmse2)
print('mae2:',score_mae2)
print('R22:',score_r22)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train.iloc[tr_idx],x_train.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model3.fit(tr_x,tr_y)
    va_pred3=model3.predict(va_x)
    score_rmse3=np.sqrt(mean_squared_error(va_y,va_pred3)).mean()
    score_mae3=mean_absolute_error(va_y,va_pred3).mean()
    score_r23=r2_score(va_y,va_pred3).mean()

In [ ]:
print('rmse3:',score_rmse3)
print('mae3:',score_mae3)
print('R23:',score_r23)

**I got RMSE 5.727 and R2 0.2329 by Ridge**

**1)-2 LightGBM**

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test)

In [ ]:
params={'metric': 'rmse'}

In [ ]:
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=5000,
                early_stopping_rounds=100,
                verbose_eval=100)

In [ ]:
predicted = gbm.predict(x_test)

In [ ]:
def pred_map(pred_df):
    rmse = np.sqrt(mean_squared_error(pred_df['data'], pred_df['pred']))
    R2 = r2_score(pred_df['data'], pred_df['pred']) 
    plt.figure(figsize=(5,5))
    ax = plt.subplot(111)
    ax.scatter('data', 'pred', data=pred_df)
    ax.set_xlabel('Data Value', fontsize=15)
    ax.set_ylabel('Pred Value', fontsize=15)
    ax.set_xlim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    ax.set_ylim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    x = np.linspace(pred_df.min().min()-0.1, pred_df.max().max()+0.1, 2)
    y = x
    ax.plot(x,y,'r-')
    plt.text(0.1, 0.9, 'rmse = {}'.format(str(round(rmse, 5))), transform=ax.transAxes, fontsize=10)
    plt.text(0.1, 0.8, 'R2 = {}'.format(str(round(R2, 5))), transform=ax.transAxes, fontsize=10)

In [ ]:
df_prediction = pd.concat([y_test.reset_index(drop=True), pd.Series(predicted)], axis=1)
df_prediction.columns = ['data', 'pred']

In [ ]:
pred_map(df_prediction)

**I got RMSE 1.71 and R2 0.59 by LightGBM**

# **4. Goals vs xG**

In [ ]:
df['goal_per']=df['Goals']/df['Matches']-df['xG']

In [ ]:
df.head()

In [ ]:
df_top = df.sort_values('goal_per', ascending=False)
df_top1=df_top.loc[:,['Name','goal_per']]

In [ ]:
df_top2=df_top1.set_index('Name')
df_top2

In [ ]:
df_top2.head(30).plot.bar()

In [ ]:
df_top2.tail(30).plot.bar()